<a href="https://colab.research.google.com/github/rilschultz/CS167Notes/blob/main/Day26_Webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS167: Day 26
## Web Scraping

#### CS167: Machine Learning, Spring 2023

Tuesday, May 2nd, 2023

📆 [Course Schedule](https://docs.google.com/spreadsheets/d/e/2PACX-1vSvFV5Mz0_YZE1d5r3gQ8IMktE4cBAsJIlP30cl2GhEpSO0J-YWV62QokSDz-OcOCsEmxMuKpY0kVlR/pubhtml?gid=0&single=true) | 🙋[PollEverywhere](https://pollev.com/meredithmoore011) | 📜 [Syllabus](https://analytics.drake.edu/~moore/cs167_s23_syllabus.html) | 📬 [CodePost Login](https://codepost.io/login)

# Admin Stuff

You should be working on:
- Project #2, due Friday, May 5th, by 11:59 pm
- Quiz #2, due Tuesday May 9th, by 11:59pm

## Load your data:

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ✨ New Material

# Web Scraping

Often times in ML, the dataset you need just doesn't exist. 

One potential way to create the dataset that you need is to use __web-scraping__: 
- The process of using scripts to extract information from existing websites. 
- Python has a great web-scraping package, `beautifulsoup`
- Another helpful package is `selenium`

## Let's Solve a Problem 🐉📚🎮

I'm not sure if you know this yet, but Professor Moore is a huge nerd. 

I love reading fantasy books, playing video games, and training machine learning models. 

The natural progression of these interests is to train a machine learning model on names from fantasy books to be able to generate my own super nerdy gamertag. 

This problem can be broken down into 2 main steps:
1. Collect a dataset of names from fantasy books.
2. Train a RNN on the dataset so that it will generate new names.

# Step #1: Webscraping a Dataset

## Brando Sando Style:

Let's scrape the names from Brandon Sanderson Characters found on [this website](https://coppermind.net/wiki/Category:Characters). 

Let's start by importing Beautiful soup and providing the URL pointing to the webpage. We use the `requests` library to get the text from the html. 

In [2]:
from bs4 import BeautifulSoup
import requests
import csv

source = requests.get('https://coppermind.net/wiki/Category:Characters').text

soup = BeautifulSoup(source, 'html.parser')

csv_file = open('fantasy_characters.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['name', 'author','genre', 'series', 'description']) 

print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Category:Characters - The Coppermind - 17th Shard
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"2f08fea2189205bad74d6d7b","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Characters","wgTitle":"Characters","wgCurRevisionId":164735,"wgRevisionId":164735,"wgArticleId":40,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Content"],"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgRelevantPageName":"Category:Characters","wgRelevantArticleId":40

Since the names are stored in `li`'s, we will use the `soup.find_all` function and provide the argument `li`. This will find all of the list items on the page. This is a good first step, but you'll notice, it's not quite what we want--it includes the list items before the actual charatcter names. 

Also, notice the line `li.text`. We are going through and getting each list item, and then only looking at the text (the `.text` line indicates this).

In [3]:
count = 0
for li in soup.find_all('li'):
  count +=1
  if count < 10: #print out the first 10
    print(li.text)

Alcatraz
Elantris & The Emperor's Soul
Legion
Mistborn series
The Reckoners
Rithmatist
Shadows for Silence in the Forests of Hell
Sixth of the Dusk
The Stormlight Archive


So now that we can get the characters's names, we want to give it the correct starting point. 

Here we'll use the `soup.find` function, and after using the 'Inspect' function on the browser, we know we're looking for the `div` element with `id='mw-pages'`. 

If we save this as `section`, we can then use this section of the website, combined with the code from above to collect a list of all of the names of Brandon Sanderson characters. 

In [4]:
section = soup.find('div', id='mw-pages')

count = 0

for li in section.find_all('li'):
  count = count + 1
  name = li.text
  
  #print the first few lines just to make sure it looks good
  if count < 10: 
    print(count, name)

  #add some other information to the csv file.
  author = 'Brandon Sanderson'
  genre = 'fantasy'
  series = ''
  description =''
  csv_writer.writerow([name, author, genre, series, description])
  

1 Aarik
2 Aaron
3 Abaray
4 Abiajan
5 Abigail Casey
6 Abraham Desjardins
7 Abrem
8 Abrial
9 Abrobadar


## Now let's add some Wheel of Time 🛞: 

New URL: https://en.wikipedia.org/wiki/List_of_Wheel_of_Time_characters 

You'll notice that the way this website is set up is a bit different from the Brando Sando website. We'll have to adjust our web-scarping code accordingly.

What should we put on line 4, `class_ = ___________` ?


In [6]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_Wheel_of_Time_characters').text

soup = BeautifulSoup(source, 'html.parser') 
section = soup.find('div', class_= 'mw-parser-output') #your code here

count = 0
for li in section.find_all('li'):

  #This one isn't quite as simple becuase we have a character name and description.
  count = count + 1
  list_item = li.text

  #not all of the entries are names, but all the names follow the syntax <name: description>  
  if(len(list_item.split(':'))>1): 
    name= list_item.split(':')[0]  #The first element is the name so we use [0]
    description = list_item.split(':')[1] #the description is after the :, so we use [1]


    # SPOILER ALERT: DONT PRINT DESCRIPTION OUT IF YOU DON'T WANT TO READ THE CHARACTER DESCRIPTIONS.
    if count < 10:
      print(count, name)

    #then we add some other information and write a row to the csv file
    series = 'Wheel of Time'
    author = 'Robert Jordan'
    genre= 'fantasy'
    csv_writer.writerow([name, author, genre, series, description])

1 Logain Ablar
2 Jonan Adley
3 Aginor
4 Dermid Ajala
5 Lelaine Akashi
6 Nalesean Aldiaya
7 Algarin Pendaloan
8 Alivia
9 Katerine Alruddin


## How about some Kingkiller Chronicles Characters
https://kingkiller.fandom.com/wiki/Category:Characters


In [34]:
source = requests.get('https://kingkiller.fandom.com/wiki/Category:Characters').text


soup = BeautifulSoup(source, 'html.parser') 
section = soup.find('div', class_= 'category-page__members')



count = 0
for li in section.find_all('li'):

  #This one isn't quite as simple becuase we have a character name and description.
  count = count + 1
  list_item = li.text.strip()

  #not all of the entries are names, but all the names follow the syntax <name: description>  
  
    # SPOILER ALERT: DONT PRINT DESCRIPTION OUT IF YOU DON'T WANT TO READ THE CHARACTER DESCRIPTIONS.
  if count > 10:
    print(count, name)

    #then we add some other information and write a row to the csv file
  series = 'Wheel of Time'
  author = 'Robert Jordan'
  genre= 'fantasy'
  


11 

 
Category
12 

 
Category
13 

 
Category
14 

 
Category
15 

 
Category
16 

 
Category
17 

 
Category
18 

 
Category
19 

 
Category
20 

 
Category
21 

 
Category
22 

 
Category
23 

 
Category
24 

 
Category
25 

 
Category
26 

 
Category
27 

 
Category
28 

 
Category
29 

 
Category
30 

 
Category
31 

 
Category
32 

 
Category
33 

 
Category
34 

 
Category
35 

 
Category
36 

 
Category
37 

 
Category
38 

 
Category
39 

 
Category
40 

 
Category
41 

 
Category
42 

 
Category
43 

 
Category
44 

 
Category
45 

 
Category
46 

 
Category
47 

 
Category
48 

 
Category
49 

 
Category
50 

 
Category
51 

 
Category
52 

 
Category
53 

 
Category
54 

 
Category
55 

 
Category
56 

 
Category
57 

 
Category
58 

 
Category
59 

 
Category
60 

 
Category
61 

 
Category
62 

 
Category
63 

 
Category
64 

 
Category
65 

 
Category
66 

 
Category
67 

 
Category
68 

 
Category
69 

 
Category
70 

 
Category
71 

 
Category
72 

 
Category
73 

 
C

## Lord of the Rings, Anyone?
Yesss, my preciousssss: https://lotr.fandom.com/wiki/Category:The_Lord_of_the_Rings_Characters

In [13]:
source = requests.get('https://en.wikipedia.org/wiki/Category:The_Lord_of_the_Rings_characters').text

# And try to do Lord of the Rings also
soup = BeautifulSoup(source, 'html.parser') 
section = soup.find('div', class_= 'mw-category mw-category-columns')



count = 0
for li in section.find_all('li'):

  #This one isn't quite as simple becuase we have a character name and description.
  count = count + 1
  list_item = li.text

  #not all of the entries are names, but all the names follow the syntax <name: description>  
  if(len(list_item.split(':'))>1): 
    name= list_item.split(':')[0]  #The first element is the name so we use [0]
    description = list_item.split(':')[1] #the description is after the :, so we use [1]


    # SPOILER ALERT: DONT PRINT DESCRIPTION OUT IF YOU DON'T WANT TO READ THE CHARACTER DESCRIPTIONS.
    if count > 10:
      print(count, name)

    #then we add some other information and write a row to the csv file
    series = 'Wheel of Time'
    author = 'Robert Jordan'
    genre= 'fantasy'
    csv_writer.writerow([name, author, genre, series, description])



AttributeError: ignored

# Annnnddd... let's give that bad boy a download

In [22]:
csv_file.close()
from google.colab import files
files.download('fantasy_characters.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Part 2: Fantasy Character Name Generator

This code should look very familiar... it's the code from Thursday, but just with a new dataset.

[Here's a link to the dataset](https://drive.google.com/file/d/1xcqFHcQ5EU4NjNP0lX6yCR3658_zuMXw/view?usp=sharing ) if you just want to download it:

In [23]:
#imports and things
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    !pip install -q -U transformers
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Import the Data
Import the data (it's a text file so it's a little bit different than a csv). 
- Then we get rid of the newlines and replace it with a space.
- we then get the vocabulary, and use a tokenizer to convert the text to sequences. 

In [25]:
from google.colab import drive
import pandas as pd
from tensorflow import keras
drive.mount('/content/drive')
#names = pd.read_csv('/content/drive/MyDrive/sanderson_names.txt',  header = None)
#names.head()

with open('/content/drive/MyDrive/datasets/sanderson_names.txt') as f:
    names = f.read()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
names = names.replace('\n'," ")
names[:100]

'Aarik Aaron Abaray Abiajan Abigail Reed Abraham Desjardins Abrial Abrobadar Abronai Abry Absence Aci'

In [27]:
# The vocabulary of our character-level language model looks like this:
"".join(sorted(set(names.lower())))

" '()-./:abcdefghijklmnopqrstuvwxyz©±√"

In [28]:
# Use Tokenizer to tokenize the Names
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(names)

In [29]:
# Embed the name 'Shallan' as tokens:
tokenizer.texts_to_sequences(["Shallan"])

[[8, 11, 2, 7, 7, 2, 5]]

In [30]:
# Revert the sequence of tokens back to the word:
tokenizer.sequences_to_texts([[8, 11, 2, 7, 7, 2, 5]])

['s h a l l a n']

In [31]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

[encoded] = np.array(tokenizer.texts_to_sequences([names])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

dataset = dataset.flat_map(lambda window: window.batch(window_length))

np.random.seed(42)
tf.random.set_seed(42)

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

dataset = dataset.prefetch(1)


for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 37) (32, 100)


# Build our model
Now we build our model. 

In [35]:
def create_model():
  model = keras.models.Sequential([
      keras.layers.GRU(64, return_sequences=True, input_shape=[None, max_id],
                      dropout=0.2),
      keras.layers.GRU(64, return_sequences=True,
                      dropout=0.2),
      keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                      activation="softmax"))
  ])
  return model


model = create_model()
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, None, 64)          19776     
                                                                 
 gru_1 (GRU)                 (None, None, 64)          24960     
                                                                 
 time_distributed (TimeDistr  (None, None, 37)         2405      
 ibuted)                                                         
                                                                 
Total params: 47,141
Trainable params: 47,141
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
#save our models during training
checkpoint_path = "training_1/fantasy_name_gen.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Now let's train our model. Notice the callbacks=[cp_callback], 
#this will save checkpoints so we can load our model later.
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=10,callbacks=[cp_callback])

Epoch 1/10
491/492 [============================>.] - ETA: 0s - loss: 2.6593
Epoch 1: saving model to training_1/fantasy_name_gen.ckpt
492/492 [==============================] - 20s 13ms/step - loss: 2.6588
Epoch 2/10
489/492 [============================>.] - ETA: 0s - loss: 2.3629
Epoch 2: saving model to training_1/fantasy_name_gen.ckpt
492/492 [==============================] - 9s 17ms/step - loss: 2.3624
Epoch 3/10
491/492 [============================>.] - ETA: 0s - loss: 2.1993
Epoch 3: saving model to training_1/fantasy_name_gen.ckpt
492/492 [==============================] - 7s 13ms/step - loss: 2.1991
Epoch 4/10
488/492 [============================>.] - ETA: 0s - loss: 2.0816
Epoch 4: saving model to training_1/fantasy_name_gen.ckpt
492/492 [==============================] - 9s 17ms/step - loss: 2.0811
Epoch 5/10
491/492 [============================>.] - ETA: 0s - loss: 1.9937
Epoch 5: saving model to training_1/fantasy_name_gen.ckpt
492/492 [==============================]

# Let's try it out!

In [37]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

# Let's pass in 'Meredi' and see what it predicts the next letter should be according to Sanderson:
X_new = preprocess(["Meredi"])

#this line takes a look at the softmax output and returns the max
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

'n'

In [38]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [39]:
tf.random.set_seed(42)

next_char("Meredi", temperature=1)

'l'

In [40]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [43]:
tf.random.set_seed(42)

print(complete_text("Ril", temperature=0.5))

Ril em mimsim millra milin mill mile milla milla mila


In [44]:
print(complete_text("t", temperature=1))

tribb winmriana jops jon (sprithhe) raokes) monkwos


In [45]:
print(complete_text("t", temperature=2))


tpari uku duqqa d'fur adiadawalid emalin rddidanaau


In [53]:
import random
for i in range(10):
  new_name = complete_text('Gar', 15, temperature=0.75)
  print(new_name.split(" ")[0].title())

Garicatatan
Gar
Gareling
Garinor
Garile
Garimer
Garlis
Gar
Gar
Gart


# Let's try loading our model

In [54]:
!pip install pyyaml h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [55]:
os.listdir(checkpoint_dir)

['fantasy_name_gen.ckpt.index',
 'checkpoint',
 'fantasy_name_gen.ckpt.data-00000-of-00001']

In [56]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_1/fantasy_name_gen.ckpt'

In [57]:
# disable warnings becuase we live dangerously:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

print(complete_text("t", temperature=0.25))

ther shaden shadry shanesh shamar shalana shalana s
